In [1]:
# Import packages for building map
import numpy as np
import pandas as pd
import sodapy as sp
import pandas_bokeh as pbk
pbk.output_notebook()
from sodapy import Socrata
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes 
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource, CustomJS,
                          CustomJSFilter, GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

Loading BokehJS ...

In [2]:
# Load file with state geometry
contiguous_cat = gpd.read_file('divisions-administratives-v1r0-comarques-1000000-20210122.shp')
cols = contiguous_cat.select_dtypes(include=[np.object]).columns
contiguous_cat[cols] = contiguous_cat[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
# Load file with each region's population
df_pobl = pd.read_excel("PoblacioComarques.xls")
# importar el nostre dataset
client = Socrata("analisi.transparenciacatalunya.cat", None)
results = client.get("rmgc-ncpb", limit=16774)
# Convert to pandas DataFrame
results_df =  gpd.GeoDataFrame.from_records(results)
contiguous_cat.head()

<ipython-input-2-dcf9fb9c1110>:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cols = contiguous_cat.select_dtypes(include=[np.object]).columns


,CODICOMAR,NOMCOMAR,geometry
0,01,Alt Camp,"POLYGON ((373020.449 4590188.741, 372482.744 4..."
1,02,Alt Emporda,"MULTIPOLYGON (((513760.140 4662388.220, 513758..."
2,03,Alt Penedes,"POLYGON ((386593.300 4595896.200, 388616.100 4..."
3,04,Alt Urgell,"MULTIPOLYGON (((360800.100 4649174.000, 360624..."
4,05,Alta Ribagorca,"POLYGON ((327485.124 4721105.796, 327436.728 4..."


In [3]:
# num d'accidents en funció de la comarca i ordenats alfabèticament respecte del nom de la comarca
com = pd.value_counts(results_df['nomcom'])
com = com.to_frame() # convertir a Data Frame
com = com.reset_index() # introduir un index (0, 1, 2...)
com = com.sort_values("index") # la columna de velocitat es diu index, ordenar-la
com = com.rename(columns={"index": "nomcom", "nomcom": "#accidents"}) # canviar els noms correctes
Comar = gpd.GeoDataFrame(com).set_index("nomcom")
Comar = Comar.reset_index()
Comar.head() #42

,nomcom,#accidents
0,Alt Camp,146
1,Alt Emporda,563
2,Alt Penedes,397
3,Alt Urgell,187
4,Alta Ribagorca,30


In [4]:
prov = results_df[['nomcom', 'nomdem']]
prov = prov.sort_values('nomcom') 
prov.head() #16mil

,nomcom,nomdem
214,Alt Camp,Tarragona
1054,Alt Camp,Tarragona
8587,Alt Camp,Tarragona
14204,Alt Camp,Tarragona
6112,Alt Camp,Tarragona


In [5]:
merge = prov.merge(Comar, left_on='nomcom', right_on='nomcom') # associem la comarca a la província, merge 16 mil
#filtrar Bcn
merge.set_index('nomdem', inplace=True)
merge_bcn=merge.loc[['Barcelona']]
#eliminar duplicats
merge_bcn = merge_bcn.drop_duplicates(subset='nomcom',keep='first')
bcn=merge_bcn.reset_index()
bcn.head() #13


,nomdem,nomcom,#accidents
0,Barcelona,Alt Penedes,397
1,Barcelona,Anoia,283
2,Barcelona,Bages,555
3,Barcelona,Baix Llobregat,1356
4,Barcelona,Barcelones,3004


In [6]:
contiguous_cat = contiguous_cat.drop({0,1,3,4,7,8,9,11,14,15,17,18,19,21,22,24,25,26,27,28,29,30,31,32,34,35,36,37,38})


In [7]:
geom = contiguous_cat[["NOMCOMAR", "geometry"]] # escollir les columnes: comarca i geometria de mcp
geom = geom.sort_values('NOMCOMAR') # ordenar alfabèticament
Geom = gpd.GeoDataFrame(geom).set_index("NOMCOMAR") # tornem a posar l'índex
Geom = Geom.reset_index()
Geom = Geom.rename(columns={"NOMCOMAR": "nomcom"}) # canviar el nom
Geom.head() #13

,nomcom,geometry
0,Alt Penedes,"POLYGON ((386593.300 4595896.200, 388616.100 4..."
1,Anoia,"POLYGON ((371426.378 4628557.172, 372106.708 4..."
2,Bages,"POLYGON ((417259.067 4646530.811, 417513.903 4..."
3,Baix Llobregat,"POLYGON ((402255.460 4605174.574, 402529.150 4..."
4,Barcelones,"MULTIPOLYGON (((432584.620 4580813.510, 432583..."


In [8]:
merge2 = prov.merge(Geom, left_on='nomcom', right_on='nomcom') # associem la comarca a la província
#filtrar Bcn
merge2.set_index('nomdem', inplace=True)
merge2_bcn=merge.loc[['Barcelona']]
#eliminar duplicats
merge2_bcn = merge2_bcn.drop_duplicates(subset='nomcom',keep='first')
bcn2=merge_bcn.reset_index()
bcn2.head() #13

,nomdem,nomcom,#accidents
0,Barcelona,Alt Penedes,397
1,Barcelona,Anoia,283
2,Barcelona,Bages,555
3,Barcelona,Baix Llobregat,1356
4,Barcelona,Barcelones,3004


In [9]:
# seleccionar columna de geometries
geometry = Geom[['geometry']]
# ordenem alfabèticament i deixem bonic el dataset de poblacions
df_pobl=df_pobl.sort_values('comarca')
df_pobl=pd.DataFrame(df_pobl).set_index('comarca')


In [10]:
df_pobl=df_pobl.reset_index()
df_pobl.head() #42

,comarca,pobl,pobl_percent
0,Alt Camp,44168,0.0059
1,Alt Emporda,136981,0.0182
2,Alt Penedes,107724,0.0143
3,Alt Urgell,20201,0.0027
4,Alta Ribagorca,3813,0.0005


In [11]:
merge_norm = prov.merge(df_pobl, left_on='nomcom', right_on='comarca') # associem la comarca a la província
#filtrar Bcn
merge_norm.set_index('nomdem', inplace=True)
merge_bcn_norm=merge_norm.loc[['Barcelona']]
#eliminar duplicats
merge_bcn_norm = merge_bcn_norm.drop_duplicates(subset='nomcom',keep='first')
bcn_norm = merge_bcn_norm.reset_index()
bcn_norm.head() #13

,nomdem,nomcom,comarca,pobl,pobl_percent
0,Barcelona,Alt Penedes,Alt Penedes,107724,0.0143
1,Barcelona,Anoia,Anoia,119129,0.0158
2,Barcelona,Bages,Bages,174703,0.0232
3,Barcelona,Baix Llobregat,Baix Llobregat,812146,0.1077
4,Barcelona,Barcelones,Barcelones,2239915,0.2969


In [12]:
# seleccionar columnes pobl i pobl_percent
norm = bcn_norm[['pobl', 'pobl_percent']]
combo = pd.concat([bcn, geometry, norm], axis=1) # juntem comarques, #accidents, geometria i poblacions de bcn
combo['accpercadamilhab_bcn'] = combo['#accidents']/combo['pobl']*1000 # afegim col per a #accidents per cada 1000 habitants
combo.head()


,nomdem,nomcom,#accidents,geometry,pobl,pobl_percent,accpercadamilhab_bcn
0,Barcelona,Alt Penedes,397,"POLYGON ((386593.300 4595896.200, 388616.100 4...",107724,0.0143,3.685344
1,Barcelona,Anoia,283,"POLYGON ((371426.378 4628557.172, 372106.708 4...",119129,0.0158,2.375576
2,Barcelona,Bages,555,"POLYGON ((417259.067 4646530.811, 417513.903 4...",174703,0.0232,3.176820
3,Barcelona,Baix Llobregat,1356,"POLYGON ((402255.460 4605174.574, 402529.150 4...",812146,0.1077,1.669651
4,Barcelona,Barcelones,3004,"MULTIPOLYGON (((432584.620 4580813.510, 432583...",2239915,0.2969,1.341122


In [13]:
#triar columnes
combo = combo[['#accidents', 'nomcom', 'geometry', 'accpercadamilhab_bcn']]
combo

,#accidents,nomcom,geometry,accpercadamilhab_bcn
0,397,Alt Penedes,"POLYGON ((386593.300 4595896.200, 388616.100 4...",3.685344
1,283,Anoia,"POLYGON ((371426.378 4628557.172, 372106.708 4...",2.375576
2,555,Bages,"POLYGON ((417259.067 4646530.811, 417513.903 4...",3.176820
3,1356,Baix Llobregat,"POLYGON ((402255.460 4605174.574, 402529.150 4...",1.669651
4,3004,Barcelones,"MULTIPOLYGON (((432584.620 4580813.510, 432583...",1.341122
5,187,Bergueda,"MULTIPOLYGON (((391614.800 4661923.500, 391606...",4.797332
6,346,Garraf,"MULTIPOLYGON (((386849.020 4561141.550, 386847...",2.359359
7,786,Maresme,"POLYGON ((477054.900 4621435.100, 478728.500 4...",1.777991
8,90,Moianès,"POLYGON ((428837.859 4639478.022, 428961.755 4...",6.751688
9,480,Osona,"MULTIPOLYGON (((418580.652 4643167.275, 418307...",3.065291


In [14]:
combo = gpd.GeoDataFrame(combo, geometry=combo['geometry'],crs='')
type(combo)

geopandas.geodataframe.GeoDataFrame

In [15]:
# Representacio interactiva
# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson = combo.to_json())

In [17]:
# Define color palettes
palette = brewer['BuGn'][8]
palette = palette[::-1] # reverse order of colors so higher values have darker colors

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 7)

# Define custom tick labels for color bar.
tick_labels = {'0': '0', '50': '500',
               '1000':'10000', '15000':'150000',
               '20000':'20000', '25000':'250000',
               '30000':'30000', '350000':'350000',
               '40000':'40000+'}

# Create color bar.
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal',
                     major_label_overrides = tick_labels)

# Create figure object.
p = figure(title = "Nombre d'accidents per cada 1000 habitants 2010-18", plot_height = 600 ,
           plot_width = 600, toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'accpercadamilhab_bcn', 'transform' : color_mapper},
                   line_color = 'gray', line_width = 0.25, fill_alpha = 1)

# Create hover tool
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('Comarca','@nomcom'),('# Accidents', '@accpercadamilhab_bcn')]))

# Specify layout
p.add_layout(color_bar, 'below')

show(p)